In [1]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import covalent as ct

In [2]:
@ct.electron
def create_data(n):
    X = np.linspace(0, 2*np.pi, n)
    X.requires_grad = False
    Y = np.sin(X)
    X_test = np.linspace(0.2, 2*np.pi+0.2, n)
    Y_test = np.sin(X_test)
    return X, Y, X_test, Y_test

In [3]:
@ct.electron
def plot_data(X_train, Y_train, X_test, Y_test):
    plt.scatter(X_train, Y_train, color='#219ebc', label='Training Data')
    plt.scatter(X_test, Y_test, color='#fb8500', alpha=0.5, label='Test Data')
    plt.legend()
    plt.savefig('input_data.png')

In [4]:
#X, Y, X_test, Y_test = create_data(5)

In [5]:
#@ct.electron
#def cost_fn(params, X):
#    #X, Y, X_test, Y_test = create_data(5)
#    predictions = [quantum_circuit(x, params)  for x in X]
#    cost = loss_func(predictions)
#    return cost

In [6]:
dev = qml.device('default.qubit', wires=1)
@qml.qnode(dev)
def quantum_circuit(datapoint, params):
    qml.RX(datapoint, wires=0)
    qml.Rot(params[0], params[1], params[2], wires=0)
    return qml.expval(qml.PauliZ(wires=0))

In [7]:
@ct.electron
def gradient_descent(X):
    X, Y, X_test, Y_test = create_data(5)
    @ct.electron
    def cost_fn(params):
        #X, Y, X_test, Y_test = create_data(5)
        predictions = [quantum_circuit(x, params)  for x in X]
        cost = loss_func(predictions)
        return cost
    opt = qml.GradientDescentOptimizer(stepsize=0.3)
    params = np.array([0.1,0.1,0.1],requires_grad=True)
    for i in range (100):
        # Over each step the parameters change to give a better cost
        params, prev_cost = opt.step_and_cost(cost_fn,params) 
        if i%10 == 0:
             # We print the result after every 10 steps
            print(f'Step = {i} Cost = {cost_fn(params)}')
    return params

In [8]:
@ct.electron
def make_predictions(X_test, params):
    test_predictions = []
    for x_test in X_test:
        prediction = quantum_circuit(x_test,params)
        test_predictions.append(prediction)
    return test_predictions

In [9]:
@ct.electron
def loss_func(predictions):
    X, Y, X_test, Y_test = create_data(5)
    # This is a postprocessing step. Here we use a least squares metric 
    # based on the predictions of the quantum circuit and the outputs 
    # of the training data points.
    total_losses = 0
    for i in range(len(Y)):
        output = Y[i]
        prediction = predictions[i]
        loss = (prediction - output)**2
        total_losses += loss
    return total_losses

In [10]:
@ct.lattice
def workflow():
    X, Y, X_test, Y_test = create_data(5)
    plot_data(X, Y, X_test, Y_test)
    params = gradient_descent(X=X)
    make_predictions(X_test=X_test, params=params)
    #params = gradient_descent(X=X)
    #return params

In [11]:
workflow.draw()

/Users/annahughes/demonstration/covalent/covalent/_workflow/lattice.py:158: UserWarning: Please make sure you are not manipulating an object inside the lattice.
  warnings.warn(


NameError: name 'params' is not defined

In [ ]:
ct.dispatch(workflow)()

In [ ]:
params = workflow()
print(params)

In [ ]:
test_predictions = []
X, Y, X_test, Y_test = create_data(5)
for x_test in X_test:
    prediction = quantum_circuit(x_test,params)
    test_predictions.append(prediction)

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.plot(X, Y, c='#219ebc', label='Training data')
ax1.plot(X_test,Y_test, c='#fb8500', label='Test data')
ax1.scatter(X_test,test_predictions, s=30, c='k', marker="x", label='Model Predicitons')
plt.xlabel("Inputs")
plt.ylabel("Outputs")
plt.title("QML results")

plt.legend(loc='upper right');
plt.show()